# 自动微分机制
torch中的核心概念是torch.tensor, 即张量。

张量的重要性质是 .requires_grad, 如果设置它的属性 .requires_grad 为 True，那么它将会追踪对于该张量的所有操作。当完成计算后可以通过调用 .backward()，来自动计算所有的梯度。这个张量的所有梯度将会自动累加到.grad属性。  对于自己定义的张量，默认.requires_grad=False, 通过torch神经网络模型生成的张量（包括网络的可训练参数）默认为True。

除此之外，还有两个性质：.grad 和 .grad_fn

(1).grad_fn是一个指向该张量创建过程的函数的引用，它指示了这个张量是通过某种操作（例如加法、矩阵乘法等）生成的。它保存着该张量的计算历史，因此可以追踪它在计算图中的前向传播过程。如果这个张量是由用户直接创建的（例如通过 torch.tensor() 创建的常量张量），它的 grad_fn 为 None。神经网络前向传播的过程中得到的输出out和损失函数等有这个属性。

(2).grad是一个张量的属性，用来存储该张量的梯度。梯度是通过反向传播计算得到的，它代表了该张量相对于某个标量值（通常是损失函数）的导数。在反向传播（backward()）后，grad 属性会存储计算得到的梯度值。通常，你会通过 tensor.grad 来访问这个梯度。如果一个张量没有被标记为需要梯度（即 requires_grad=False），或者没有进行反向传播，grad 就会是 None。

当然，可以阻止计算被追踪。除了将.requries_grad设置为False外，可以通过将代码块包装在 with torch.no_grad(): 中，来阻止 autograd 跟踪设置了.requires_grad=True的张量的历史记录。


在计算图中，叶子节点是指没有父节点的节点，通常它们是直接由用户创建的张量。简单来说，叶子节点就是“最初的”数据来源。这些节点通常是模型的输入张量、权重或者偏置。

反向传播时，会根据中间张量和模型输出等张量的.grad_fn属性计算生成.grad

In [8]:
import os
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim

In [2]:
net = nn.Sequential(nn.Linear(20, 256), nn.ReLU(), nn.Linear(256, 1))

In [3]:
for name, param in net.named_parameters():
    print(f"Layer: {name}, Shape: {param.shape}, {param.requires_grad}, {param.grad_fn}, {param.grad}")

Layer: 0.weight, Shape: torch.Size([256, 20]), True, None, None
Layer: 0.bias, Shape: torch.Size([256]), True, None, None
Layer: 2.weight, Shape: torch.Size([1, 256]), True, None, None
Layer: 2.bias, Shape: torch.Size([1]), True, None, None


In [9]:
# 定义优化器（使用随机梯度下降 SGD）
optimizer = optim.SGD(net.parameters(), lr=0.01)  # lr 是学习率

# 生成随机输入（batch_size=10, 输入特征维度=20）
input_data = torch.randn(10, 20)  # 10个样本，每个样本20维

# 前向传播
output = net(input_data)  # 计算输出

# 定义损失函数（假设是回归任务，使用均方误差损失）
loss_fn = nn.MSELoss()
target = torch.randn(10, 1)  # 随机生成目标值
loss = loss_fn(output, target)  # 计算损失

# 反向传播
optimizer.zero_grad()  # 清空之前的梯度
loss.backward()        # 计算梯度，等价于记录所有的d loss/d param

# 更新网络参数
optimizer.step()       # 更新参数，等价于param=param-lr*param.grad

In [10]:
# 模型参数的 grad_fn 是 None，因为它们是叶子节点。
for name, param in net.named_parameters():
    print(f"Layer: {name}, Shape: {param.shape}, {param.requires_grad}, {param.grad_fn}, {param.grad}")

Layer: 0.weight, Shape: torch.Size([256, 20]), True, None, tensor([[-7.5427e-04,  1.0441e-02, -2.5685e-03,  ...,  4.4088e-03,
          7.9382e-03,  5.0865e-03],
        [ 5.0582e-04, -5.0807e-03,  2.9341e-03,  ...,  3.5758e-05,
         -4.7831e-03, -1.1671e-03],
        [ 1.8860e-03,  7.7030e-04,  9.3196e-04,  ...,  1.6744e-03,
          5.4542e-04,  1.2665e-03],
        ...,
        [ 8.1465e-03, -9.5549e-03, -4.8032e-04,  ...,  2.8421e-03,
         -1.2170e-02, -3.8942e-03],
        [-3.6834e-03, -3.0086e-03,  1.5137e-03,  ...,  1.8260e-03,
          4.2130e-03, -3.5719e-03],
        [-4.8988e-03, -4.7181e-03,  7.1656e-03,  ...,  5.4587e-03,
          2.2547e-03,  7.0951e-03]])
Layer: 0.bias, Shape: torch.Size([256]), True, None, tensor([ 5.5503e-03, -4.3952e-03, -3.0501e-03, -2.7104e-04,  2.6061e-03,
         3.6497e-03, -2.2752e-06,  4.7337e-04, -2.1895e-03,  4.3336e-05,
        -6.8732e-03, -4.4909e-03,  5.7705e-03,  2.0965e-04,  2.8352e-04,
        -1.3214e-02,  5.2679e-03,  9.

In [ ]:
A = torch.tensor([2.0], requires_grad=True)

In [ ]:
A.requires_grad

True

In [ ]:
A.grad_fn

In [ ]:
B=A**2

In [ ]:
B.requires_grad

True

In [ ]:
B.grad_fn